<a href="https://colab.research.google.com/github/arturbernardo/data_sus/blob/main/obitos_cid_datasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd

In [57]:
df = pd.read_csv('/content/sim_obitos.csv', delimiter=';')

In [51]:
df

,data_extracao,identificador_pessoa2,obito,nascimento,escolaridade,sexo,raca,gestacao,gravidez,tipo_obito,circunstancia_obito,cid_causa_basica,obito_prematuro,obito_mulher_idade_fertil
0,2025-01-01,NaN,12/2017,12/1935,COM 12 ANOS OU MAIS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,NaN,I259,NÃO,NÃO
1,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2018,06/1949,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,K703,NÃO,NÃO
2,2025-01-01,NaN,12/2017,06/1979,COM 12 ANOS OU MAIS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,IGNORADO,Y209,NÃO,NÃO
3,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,03/2021,12/1959,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,I350,SIM,NÃO
4,2025-01-01,NaN,03/2016,02/1960,NaN,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,C679,SIM,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135543,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,06/2017,01/1952,DE 4 A 7 ANOS,MASCULINO,PRETA,NaN,NaN,NÃO FETAL,NaN,E149,SIM,NÃO
135544,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2021,12/1973,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,G409,NÃO,NÃO
135545,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2021,09/1957,DE 8 A 11 ANOS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,NaN,C211,SIM,NÃO
135546,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2021,02/1949,IGNORADO,FEMININO,PRETA,NaN,NaN,NÃO FETAL,NaN,C349,NÃO,NÃO


In [58]:
CID = df['cid_causa_basica'].unique()
CID

array(['I259', 'K703', 'Y209', ..., 'X956', 'F402', 'P229'], dtype=object)

In [60]:
cid_subcat = pd.read_csv('/content/CID-10-SUBCATEGORIAS.CSV', encoding='latin-1', delimiter=';')

In [61]:
cid_subcat

,SUBCAT,CLASSIF,RESTRSEXO,CAUSAOBITO,DESCRICAO,DESCRABREV,REFER,EXCLUIDOS,Unnamed: 8
0,A000,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00.0 Colera dev Vibrio cholerae 01 biot cholerae,NaN,NaN,NaN
1,A001,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00.1 Colera dev Vibrio cholerae 01 biot El Tor,NaN,NaN,NaN
2,A009,NaN,NaN,NaN,Cólera não especificada,A00.9 Colera NE,NaN,NaN,NaN
3,A010,NaN,NaN,NaN,Febre tifóide,A01.0 Febre tifoide,NaN,NaN,NaN
4,A011,NaN,NaN,NaN,Febre paratifóide A,A01.1 Febre paratifoide A,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12446,U818,NaN,NaN,NaN,Agente resistente a outros antibióticos relaci...,U81.8 Agente resist outr antibiót relac vancom...,NaN,NaN,NaN
12447,U88,NaN,NaN,NaN,Agente resistente a múltiplos antibióticos,U88 Agente resistente a múltiplos antibióticos,NaN,NaN,NaN
12448,U898,NaN,NaN,NaN,Agente resistente a outro antibiótico especifi...,U89.8 Agente resist outro antibiótico espec único,NaN,NaN,NaN
12449,U899,NaN,NaN,NaN,Agente resistente a antibiótico não especificado,U89.9 Agente resistente a antibiótico NE,NaN,NaN,NaN


In [62]:
df_index_cid = pd.read_csv('/content/sim_obitos.csv', index_col='cid_causa_basica', delimiter=';')

In [91]:
# print(cid_subcat[cid_subcat['SUBCAT'].str.startswith('X20')])
X20 = df_index_cid[df_index_cid.index.astype(str).get_level_values('cid_causa_basica').str.startswith('X')]
X20

,data_extracao,identificador_pessoa2,obito,nascimento,escolaridade,sexo,raca,gestacao,gravidez,tipo_obito,circunstancia_obito,obito_prematuro,obito_mulher_idade_fertil
cid_causa_basica,,,,,,,,,,,,,
X954,2025-01-01,NaN,12/2017,02/1939,DE 4 A 7 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,HOMICÍDIO,NÃO,NÃO
X954,2025-01-01,NaN,12/2017,05/1999,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,HOMICÍDIO,NÃO,NÃO
X959,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,03/2021,03/1998,DE 8 A 11 ANOS,MASCULINO,PARDA,NaN,NaN,NÃO FETAL,HOMICÍDIO,NÃO,NÃO
X999,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,03/2018,02/1993,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,HOMICÍDIO,NÃO,NÃO
X954,2025-01-01,NaN,12/2017,06/1996,DE 4 A 7 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,HOMICÍDIO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
X091,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,11/2017,10/1984,NaN,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,ACIDENTE,NÃO,NÃO
X091,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,11/2017,10/1984,NaN,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,ACIDENTE,NÃO,NÃO
X704,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,10/2021,12/1988,DE 4 A 7 ANOS,MASCULINO,PRETA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO


In [ ]:
df_filtrado = cid_subcat[cid_subcat['SUBCAT'].isin(X20.index)]
df_filtrado

In [105]:
df_sui = df_index_cid[df_index_cid['circunstancia_obito'] == 'SUICÍDIO']
df_sui = df_sui[df_sui['obito'].str.endswith('17')]
df_sui

,data_extracao,identificador_pessoa2,obito,nascimento,escolaridade,sexo,raca,gestacao,gravidez,tipo_obito,circunstancia_obito,obito_prematuro,obito_mulher_idade_fertil
cid_causa_basica,,,,,,,,,,,,,
X700,2025-01-01,NaN,10/2017,08/1988,DE 8 A 11 ANOS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,SIM
X700,2025-01-01,NaN,12/2017,08/1963,IGNORADO,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
X710,2025-01-01,NaN,12/2017,03/1952,COM 12 ANOS OU MAIS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
X740,2025-01-01,NaN,12/2017,03/1974,COM 12 ANOS OU MAIS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
X740,2025-01-01,NaN,09/2017,08/1992,COM 12 ANOS OU MAIS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
X700,2025-01-01,NaN,02/2017,05/1960,DE 4 A 7 ANOS,MASCULINO,PRETA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
X800,2025-01-01,NaN,02/2017,10/1992,COM 12 ANOS OU MAIS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO
X700,2025-01-01,NaN,03/2017,04/1986,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,SUICÍDIO,NÃO,NÃO


In [103]:
Z634 = df_index_cid[df_index_cid.index.astype(str).get_level_values('cid_causa_basica').str.startswith('Z634')]
Z634

,data_extracao,identificador_pessoa2,obito,nascimento,escolaridade,sexo,raca,gestacao,gravidez,tipo_obito,circunstancia_obito,obito_prematuro,obito_mulher_idade_fertil
cid_causa_basica,,,,,,,,,,,,,


In [133]:
df_index_cid['nascimento_fake_day'] = '01/' + df_index_cid['nascimento'].astype(str)

df_index_cid['nasc_date'] = pd.to_datetime(df_index_cid['nascimento_fake_day'], errors='coerce', format="%d/%m/%Y")
df_index_cid['nasc_year'] = pd.DatetimeIndex(df_index_cid['nasc_date']).year
df_index_cid

,data_extracao,identificador_pessoa2,obito,nascimento,escolaridade,sexo,raca,gestacao,gravidez,tipo_obito,circunstancia_obito,obito_prematuro,obito_mulher_idade_fertil,nasc_date,obito_fake_day,nascimento_fake_day,nasc_date_1,nasc_year
cid_causa_basica,,,,,,,,,,,,,,,,,,
I259,2025-01-01,NaN,12/2017,12/1935,COM 12 ANOS OU MAIS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,NaN,NÃO,NÃO,1935-12-01,NaN,01/12/1935,NaT,1935.0
K703,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2018,06/1949,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,NÃO,NÃO,1949-06-01,NaN,01/06/1949,NaT,1949.0
Y209,2025-01-01,NaN,12/2017,06/1979,COM 12 ANOS OU MAIS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,IGNORADO,NÃO,NÃO,1979-06-01,NaN,01/06/1979,NaT,1979.0
I350,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,03/2021,12/1959,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,SIM,NÃO,1959-12-01,NaN,01/12/1959,NaT,1959.0
C679,2025-01-01,NaN,03/2016,02/1960,NaN,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,SIM,NÃO,1960-02-01,NaN,01/02/1960,NaT,1960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E149,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,06/2017,01/1952,DE 4 A 7 ANOS,MASCULINO,PRETA,NaN,NaN,NÃO FETAL,NaN,SIM,NÃO,1952-01-01,NaN,01/01/1952,NaT,1952.0
G409,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2021,12/1973,DE 8 A 11 ANOS,MASCULINO,BRANCA,NaN,NaN,NÃO FETAL,NaN,NÃO,NÃO,1973-12-01,NaN,01/12/1973,NaT,1973.0
C211,2025-01-01,68a4cfa2b1f6db5e762dd8287425fb52,12/2021,09/1957,DE 8 A 11 ANOS,FEMININO,BRANCA,NaN,NaN,NÃO FETAL,NaN,SIM,NÃO,1957-09-01,NaN,01/09/1957,NaT,1957.0
